Refresh credentials and load the Weaviate IP. 

If the credentials expire, run this cell again to refresh them.

In [1]:
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()

%store -r WEAVIATE_IP

# Working with Weaviate

![Weaviate architecture](images/weaviate-explainer-architecture.png)

Connect to a Weaviate instance

In [ ]:
import weaviate

client = weaviate.connect_to_local(
    # STUDENT TODO:
    # Add WEAVIATE_IP as the first positional argument
    # BEGIN_SOLUTION
    WEAVIATE_IP,
    # END_SOLUTION
    # These are the AWS credentials for Weaviate to use
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }
)

client.is_ready()

True

## Create collection with vectorizer

Create a collection for financial articles with separate embeddings for title and content.

[Docs - Collection configuration](https://weaviate.io/developers/weaviate/manage-data/collections)

![Weaviate collections](images/weaviate-explainer-collections.png)

In [3]:
from weaviate.classes.config import Configure, Property, DataType

# Delete collection if it exists
if client.collections.exists("FinancialArticles"):
    client.collections.delete("FinancialArticles")

# Create collection with named vectors for title and content
client.collections.create(
    name="FinancialArticles",
    # Define property schema
    properties=[
        Property(name="article_title", data_type=DataType.TEXT),
        # STUDENT TODO:
        # Add two additional properties - "article" and "url"
        # Both should be of data_type DataType.TEXT
        # BEGIN_SOLUTION
        Property(name="article", data_type=DataType.TEXT),
        Property(name="url", data_type=DataType.TEXT),
        # END_SOLUTION
    ],
    # Configure separate embeddings for title and article content
    vector_config=[
        Configure.Vectors.text2vec_aws(
            name="title",
            source_properties=["article_title"],
            region="us-east-1",
            service="bedrock",
            model="amazon.titan-embed-text-v2:0"
        ),
        # STUDENT TODO:
        # Add a second vector configuration
        # using the same region, service, and model as above
        # Change the name to "content"
        # Change source properties to "article"
        # BEGIN_SOLUTION
        Configure.Vectors.text2vec_aws(
            name="content",
            source_properties=["article"],
            region="us-east-1",
            service="bedrock",
            model="amazon.titan-embed-text-v2:0"
        )
        # END_SOLUTION
    ],
)

## Load financial articles data

Import the financial news dataset using batch processing for efficient loading.

In [4]:
import pandas as pd
from tqdm import tqdm
from weaviate.util import generate_uuid5

# Load the financial articles dataset
df = pd.read_parquet("data/fin_news_articles_5000.parquet")
df = df[:2000]

print(f"Loaded {len(df)} articles")
print("Sample data:")
df.head()

Loaded 2000 articles
Sample data:


,date,article_title,stock_symbol,url,article
0,2022-08-04 00:00:00 UTC,Ansys Q2 22 Earnings Conference Call At 8:30 A...,ANSS,https://www.nasdaq.com/articles/ansys-q2-22-ea...,(RTTNews) - Ansys Inc. (ANSS) will host a conf...
1,2012-11-26 00:00:00 UTC,Corrections Corporation of America (CXW) Ex-Di...,PAYX,https://www.nasdaq.com/articles/corrections-co...,Corrections Corporation of America ( CXW ) has...
2,2023-07-14 00:00:00 UTC,"Technology Sector Update for 07/14/2023: ASML,...",ASML,https://www.nasdaq.com/articles/technology-sec...,"Tech stocks were lower late Friday, with the T..."
3,2020-02-16 00:00:00 UTC,3 Best Biotech Stocks to Buy for the Next Decade,VRTX,https://www.nasdaq.com/articles/3-best-biotech...,Let me start out by acknowledging that predict...
4,2021-08-24 00:00:00 UTC,iShares U.S. Transportation ETF Experiences Bi...,CSX,https://www.nasdaq.com/articles/ishares-u.s.-t...,Looking today at week-over-week shares outstan...


In [5]:
# Get collection reference
# STUDENT TODO:
# Assign client.collections.use("FinancialArticles") to variable 'articles'
# BEGIN_SOLUTION
articles = client.collections.use("FinancialArticles")
# END_SOLUTION

In [6]:
# Import data with batch processing
with articles.batch.fixed_size(batch_size=100) as batch:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        # Create object from dataframe row
        obj = {
            "article_title": row["article_title"],
            "article": row["article"],
            "url": row["url"] if "url" in row else ""
        }

        # Generate UUID to prevent duplicates
        uuid = generate_uuid5(row["article_title"] + str(row.get("url", "")))

        # STUDENT TODO:
        # Add the object to the batch
        # using batch.add_object
        # with properties=obj, uuid=uuid
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj,
            uuid=uuid
        )
        # END_SOLUTION

        # Check for errors during import
        if batch.number_errors > 10:
            print("Too many errors during import")
            break

print(f"\nImported {len(articles)} articles")

100%|██████████| 2000/2000 [00:27<00:00, 72.82it/s]



Imported 2000 articles


## Check for import errors

In [7]:
if len(articles.batch.failed_objects) > 0:
    print(f"Import completed with {len(articles.batch.failed_objects)} errors")
    for err in articles.batch.failed_objects[:5]:  # Show first 5 errors
        print(err)
else:
    print("Import completed successfully with no errors")

Import completed successfully with no errors


## Verify the data

Check that articles were imported correctly with embeddings.

In [8]:
# Check collection size
# STUDENT TODO:
# Use the built-in len() function on the 'articles' collection to get the total number of articles
# BEGIN_SOLUTION
print(f"Total articles in collection: {len(articles)}")
# END_SOLUTION

Total articles in collection: 2000


In [ ]:
# View sample data
# STUDENT TODO:
# Fetch 3 articles
# Use articles.query.fetch_objects with `limit=3``
# BEGIN_SOLUTION
response = articles.query.fetch_objects(limit=3)
# END_SOLUTION

# Inspect the response
for item in response.objects:
    print(f"Title: {item.properties['article_title']}")
    print(f"Content preview: {item.properties['article'][:100]}...\n")

Title: Here's Why Galapagos NV Is Skyrocketing 18.3% Today
Content preview: What happened
After the company reported that its oral rheumatoid arthritis drug, filgotinib, succes...

Title: 3 Things Every Investor Needs To Know
Content preview: InvestorPlace - Stock Market News, Stock Advice & Trading Tips
Flying an airplane is one of my passi...

Title: How one large bull is playing Marvell
Content preview: One investor expects a springtime rally for Marvell Technology, which is off to a strong start in 20...



## Test basic search

Quick test to verify the collection works with semantic search.

In [ ]:
# Try semantic search on titles
# STUDENT TODO:
# Use `.query.near_text` to search for articles related to "technology earnings"
# Supply the target_vector argument as "title"
# Limit results to 3
# BEGIN_SOLUTION
response = articles.query.near_text(
    query="technology earnings",
    target_vector="title",
    limit=3
)
# END_SOLUTION


print("Search results for 'technology earnings':")
# Inspect the response
# STUDENT TODO:
# BEGIN_SOLUTION
for item in response.objects:
    print(f"Title: {item.properties['article_title']}")
    print(f"Content preview: {item.properties['article'][:100]}...\n")
# END_SOLUTION

Search results for 'technology earnings':
Title: Align Technology (ALGN) Q3 Earnings Lag Estimates, Margins Drop
Content preview: Align Technology, Inc.’s ALGN third-quarter 2022 earnings per share (EPS) were $1.36, reflecting a 5...

Title: Case Builds for Tech Dividends
Content preview: It’s not a stretch to say a generation, perhaps two, of investors are hardwired to believe technolog...

Title: Technology Sector Update for 09/01/2017: MSFT, AAPL, IBM, CSCO, GOOGL, PANW, QCOM, AMBA
Content preview: Top Technology Stocks:
MSFT: +0.04%
AAPL: +0.4%
IBM: +0.4%
CSCO: +0.4%
GOOGL: +0.3%
Technology share...



## Close the client

Always close your connection when finished.

In [11]:
# Close the client after use
client.close()